In [1]:
# =========================
# 0) Imports
# =========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    import gurobipy as gp
    from gurobipy import GRB
except Exception as e:
    raise RuntimeError(
        "This notebook cell requires gurobipy + a working Gurobi license.\n"
        f"Import error: {repr(e)}"
    )

# -------------------------
# Helpers
# -------------------------
def pw_factor(r: float, H: int) -> float:
    """Present-worth factor P/A(r,H)."""
    if H <= 0:
        return 0.0
    if abs(r) < 1e-12:
        return float(H)
    return (1.0 - (1.0 + r) ** (-H)) / r

def as_2d(a):
    a = np.asarray(a)
    if a.ndim == 1:
        return a[None, :]
    return a


In [2]:
def build_hourly_profile_from_month_hour_csv(
    csv_path: str = "gemini_avg_power_month_hour_2017_2019_long.csv",
    month_col: str = "month",
    hour_col: str = "hour",
    value_col: str = "avg_power",
    base_year: int = 2021,     # non-leap year => 8760 hours
    tz: str = "UTC",
) -> np.ndarray:
    """
    Converts a (month, hour)->value table (288 rows) into a chronological 8760-hour profile.

    Returns
    -------
    profile : np.ndarray shape (8760,)
        Hourly values for a typical year.
    """
    df = pd.read_csv(csv_path)

    required = {month_col, hour_col, value_col}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in {csv_path}: {missing}. Found: {list(df.columns)}")

    df = df[[month_col, hour_col, value_col]].copy()
    df[month_col] = df[month_col].astype(int)
    df[hour_col] = df[hour_col].astype(int)
    df[value_col] = df[value_col].astype(float)

    # Ensure full 12x24 coverage
    all_pairs = pd.MultiIndex.from_product(
        [range(1, 13), range(0, 24)],
        names=[month_col, hour_col]
    )
    df = df.set_index([month_col, hour_col]).reindex(all_pairs)

    if df[value_col].isna().any():
        missing_pairs = df[df[value_col].isna()].index.tolist()[:10]
        raise ValueError(
            f"Incomplete month-hour table in {csv_path}. Missing examples: {missing_pairs}. "
            "Need all 12*24=288 combinations."
        )

    # Create hourly timestamps for a non-leap year
    idx = pd.date_range(
        start=f"{base_year}-01-01",
        end=f"{base_year+1}-01-01",
        freq="h",
        inclusive="left",
        tz=tz
    )
    if len(idx) != 8760:
        raise RuntimeError(f"Expected 8760 hours, got {len(idx)} for base_year={base_year}.")

    tmp = pd.DataFrame({"ts": idx})
    tmp["month"] = tmp["ts"].dt.month
    tmp["hour"] = tmp["ts"].dt.hour

    lut = df.reset_index().rename(columns={value_col: "val"})
    tmp = tmp.merge(lut, on=["month", "hour"], how="left")

    if tmp["val"].isna().any():
        raise RuntimeError("Mapping month-hour -> hourly series failed (NaNs after merge).")

    profile = tmp["val"].to_numpy(dtype=float)
    profile = np.clip(profile, 0.0, None)
    return profile

In [3]:
# =========================
# 1) Time series + scenarios (historical day-ahead prices)
#    Build scenarios from full years (2020–2025) OR a 1-week test window.
# =========================
import numpy as np
import pandas as pd

dt = 1.0
hours_per_year = 8760.0

price_path = "day_ahead_prices_2020_2025_combined.csv"
ts_col = "timestamp"
price_col = "price"

# -------------------------
# TEST SWITCH (turn on/off)
# -------------------------
TEST_MODE = False  # <--- set True to test on a single week
TEST_START_UTC = "2022-01-03 00:00:00"  # inclusive, UTC
TEST_DAYS = 7  # 7 days = 168 hours
TEST_TILE_TO_YEAR = False  # set True to repeat the week to 8760 hours

dfp = pd.read_csv(price_path)
dfp[ts_col] = pd.to_datetime(dfp[ts_col], utc=True, errors="coerce")
dfp = dfp.dropna(subset=[ts_col, price_col]).sort_values(ts_col)

# If testing, cut down the raw data early (faster than processing everything)
if TEST_MODE:
    start_ts = pd.Timestamp(TEST_START_UTC, tz="UTC")
    end_ts = start_ts + pd.Timedelta(days=TEST_DAYS)
    dfp = dfp[(dfp[ts_col] >= start_ts) & (dfp[ts_col] < end_ts)]

dfp = dfp.set_index(ts_col)

# Ensure hourly grid (handles missing values robustly)
price_series = (
    dfp[price_col].astype(float)
    .resample("h").mean()
    .interpolate("time")
)

if TEST_MODE:
    # Ensure exact week slice on the hourly series
    start_ts = pd.Timestamp(TEST_START_UTC, tz="UTC")
    end_ts = start_ts + pd.Timedelta(days=TEST_DAYS)
    price_series = price_series.loc[start_ts : end_ts - pd.Timedelta(hours=1)]

    if len(price_series) != 24 * TEST_DAYS:
        raise ValueError(
            f"Test window does not have exactly {24*TEST_DAYS} hourly points; got {len(price_series)}. "
            "Pick a different TEST_START_UTC with full hourly coverage."
        )

    if TEST_TILE_TO_YEAR:
        reps = int(hours_per_year // len(price_series))
        rem = int(hours_per_year - reps * len(price_series))
        tiled = np.concatenate([np.tile(price_series.to_numpy(float), reps),
                                price_series.to_numpy(float)[:rem]])
        price_series = pd.Series(tiled)  # index not needed for the optimization horizon

# Drop Feb 29 only in normal mode so each year has exactly 8760 hours
if not TEST_MODE:
    is_feb29 = (price_series.index.month == 2) & (price_series.index.day == 29)
    price_series = price_series[~is_feb29]

# -------------------------
# Build scenarios
# -------------------------
if TEST_MODE and not TEST_TILE_TO_YEAR:
    # One scenario = the week (T=168)
    prices = price_series.to_numpy(dtype=float)[None, :]  # shape (1,T)
    kept_years = [f"TEST_WEEK_FROM_{TEST_START_UTC}"]
else:
    # Each full calendar year = one scenario (T=8760)
    all_years = sorted(price_series.index.year.unique())

    prices_list = []
    kept_years = []
    for y in all_years:
        ys = price_series[price_series.index.year == y]
        if len(ys) == 8760:
            prices_list.append(ys.to_numpy(dtype=float))
            kept_years.append(y)

    if len(prices_list) == 0:
        raise ValueError("No full 8760-hour years found. Check data coverage and timestamps.")

    prices = np.vstack(prices_list)  # shape (S,T)

S, T = prices.shape
year_scale = hours_per_year / (T * dt)  # =1.0 in normal mode; >1.0 in week-only test
scenario_prob = np.ones(S, dtype=float) / S

print(f"Loaded scenarios from: {kept_years}")
print(f"prices shape: (S,T)=({S},{T}), year_scale={year_scale}")
print(f"price stats: min={prices.min():.2f}, mean={prices.mean():.2f}, max={prices.max():.2f}")

def make_surplus_proxy_from_prices(prices_ST, surplus_max_mw=50.0, low_quantile=0.20):
    thresh = np.quantile(prices_ST, low_quantile, axis=1, keepdims=True)
    return np.where(prices_ST <= thresh, surplus_max_mw, 0.0).astype(float)

surplus = make_surplus_proxy_from_prices(prices, surplus_max_mw=50.0, low_quantile=0.20)

# =========================
# 1b) Surplus (Gemini production profile -> hourly surplus)
# =========================

gemini_power_path = "gemini_avg_power_month_hour_2017_2019_long.csv"  # your attached file name

gemini_profile = build_hourly_profile_from_month_hour_csv(
    gemini_power_path,
    month_col="month",
    hour_col="hour",
    value_col="avg_power",
    base_year=2021,
    tz="UTC",
)

# --- Align Gemini profile to the optimization horizon T ---
gemini_profile = np.asarray(gemini_profile, dtype=float).ravel()

def _hour_of_year_no_feb29(ts_utc: pd.Timestamp) -> int:
    """
    Returns the hour index (0..8759) of ts within its year, counting hourly steps,
    while skipping Feb 29 to stay consistent with the 8760-hour convention.
    """
    if ts_utc.tz is None:
        ts_utc = ts_utc.tz_localize("UTC")
    y0 = pd.Timestamp(f"{ts_utc.year}-01-01 00:00:00", tz="UTC")
    rng = pd.date_range(y0, ts_utc, freq="h", inclusive="left")
    rng = rng[~((rng.month == 2) & (rng.day == 29))]
    return len(rng)

def align_profile_to_T(profile_1d: np.ndarray, T: int, start_hour: int = 0) -> np.ndarray:
    profile_1d = np.asarray(profile_1d, dtype=float).ravel()
    n = len(profile_1d)

    if n == T:
        return profile_1d

    # Common case: full-year profile but shorter test horizon -> slice from start_hour
    if n == 8760 and T < 8760:
        start_hour = int(start_hour) % 8760
        end = start_hour + T
        if end <= 8760:
            return profile_1d[start_hour:end]
        else:
            # wrap around year-end
            return np.concatenate([profile_1d[start_hour:], profile_1d[:end - 8760]])

    # If profile shorter than T -> tile up to T
    if n < T:
        reps = int(np.ceil(T / n))
        return np.tile(profile_1d, reps)[:T]

    # If profile longer than T -> truncate
    return profile_1d[:T]

# Choose start hour for slicing in TEST_MODE (if available)
start_hour = 0
if "TEST_MODE" in globals() and TEST_MODE and "TEST_START_UTC" in globals():
    start_ts = pd.Timestamp(TEST_START_UTC, tz="UTC")
    start_hour = _hour_of_year_no_feb29(start_ts)

gemini_profile = align_profile_to_T(gemini_profile, T, start_hour=start_hour)

# Final safety check
if len(gemini_profile) != T:
    raise ValueError(f"Gemini profile length {len(gemini_profile)} does not match T={T} after alignment.")

# Interpretation choice:
# surplus[t] = available power that can be used to charge storage (MW).
# If you assume you can divert a fraction of Gemini output to storage:
curtailment_fraction = 1.0   # set e.g. 0.3 if only 30% is realistically available
surplus = np.tile((curtailment_fraction * gemini_profile)[None, :], (S, 1))

# Optional: add mild year-to-year variability (keeps same shape but different per scenario)
rng = np.random.default_rng(42)
year_sigma = 0.10  # 10% multiplicative variation
year_scale_factors = rng.normal(1.0, year_sigma, size=(S, 1))
surplus = np.clip(surplus * year_scale_factors, 0.0, None)

print(f"surplus shape: {surplus.shape}, surplus stats: min={surplus.min():.2f}, mean={surplus.mean():.2f}, max={surplus.max():.2f}")

Loaded scenarios from: [2020, 2021, 2022, 2023, 2024]
prices shape: (S,T)=(5,8760), year_scale=1.0
price stats: min=-500.00, mean=110.10, max=872.96
surplus shape: (5, 8760), surplus stats: min=239.10, mean=408.54, max=568.30


In [4]:
# =========================
# 2) Technology data (PLACEHOLDERS: fill with your estimates)
#    Subsidy structures reflect what you already collected.
# =========================

# Convert doc values:
# Hydropower subsidy example: 0.1693 €/kWh in 2024 => 169.3 €/MWh (15 years)
hydro_sub_eur_per_MWh = 0.1693 * 1000.0

# Hydrogen electrolysis subsidy example: 0.3796 €/kWh_HHV (15 years)
# We'll apply it to produced H2_HHV = eta_ch * charged_electricity
h2_sub_eur_per_MWh_HHV = 0.3796 * 1000.0

TECH = {
    "PHS": {
        "eta_ch": 0.90,
        "eta_dis": 0.90,
        "self_dis": 0.0,             # per step fraction
        "Emax": 5000.0,              # MWh_store
        "Pch_max": 500.0,            # MW
        "Pdis_max": 500.0,           # MW
        "capex_E": 150000.0,         # €/MWh_store (placeholder)
        "capex_Pch": 50000.0,        # €/MW (placeholder)
        "capex_Pdis": 50000.0,       # €/MW (placeholder)
        "fom_E": 2000.0,             # €/MWh_store/year (placeholder)
        "fom_P": 5000.0,             # €/MW/year (placeholder)
        "vom_ch": 0.5,               # €/MWh_e (placeholder)
        "vom_dis": 0.5,              # €/MWh_e (placeholder)
        "subsidy_on_discharge_eur_per_MWh_e": hydro_sub_eur_per_MWh,
        "subsidy_on_h2_MWh_HHV": 0.0,
        "capex_grant_per_MWh_store": 0.0,
        "capex_grant_cap_total": 0.0,
        "capex_discount_factor": 0.0,  # e.g., 0.10 for ~10% EIA proxy
        "salvage_frac": 0.10,
    },
    "Flywheel": {
        "eta_ch": 0.92,
        "eta_dis": 0.92,
        "self_dis": 0.001,           # demo
        "Emax": 200.0,
        "Pch_max": 200.0,
        "Pdis_max": 200.0,
        "capex_E": 400000.0,         # placeholder
        "capex_Pch": 80000.0,        # placeholder
        "capex_Pdis": 80000.0,       # placeholder
        "fom_E": 6000.0,             # placeholder
        "fom_P": 8000.0,             # placeholder
        "vom_ch": 1.0,
        "vom_dis": 1.0,
        "subsidy_on_discharge_eur_per_MWh_e": 0.0,
        "subsidy_on_h2_MWh_HHV": 0.0,
        "capex_grant_per_MWh_store": 0.0,
        "capex_grant_cap_total": 0.0,
        "capex_discount_factor": 0.10,  # EIA proxy from your notes
        "salvage_frac": 0.15,
    },
    "Battery": {
        "eta_ch": 0.95,
        "eta_dis": 0.95,
        "self_dis": 0.0005,
        "Emax": 1000.0,
        "Pch_max": 500.0,
        "Pdis_max": 500.0,
        "capex_E": 250000.0,         # placeholder
        "capex_Pch": 60000.0,        # placeholder
        "capex_Pdis": 60000.0,       # placeholder
        "fom_E": 5000.0,             # placeholder
        "fom_P": 7000.0,             # placeholder
        "vom_ch": 2.0,
        "vom_dis": 2.0,
        "subsidy_on_discharge_eur_per_MWh_e": 0.0,
        "subsidy_on_h2_MWh_HHV": 0.0,
        # SPRILA example (big company): 70 €/kWh => 70,000 €/MWh (cap applies)
        "capex_grant_per_MWh_store": 70000.0,   # €/MWh_store
        "capex_grant_cap_total": 350000.0,      # € cap (from your doc table)
        "capex_discount_factor": 0.10,          # EIA proxy if you apply it
        "salvage_frac": 0.10,
    },
    "Hydrogen": {
        # Interpret SOC as stored H2 energy (MWh_HHV).
        # eta_ch: electricity -> H2_HHV conversion
        # eta_dis: H2_HHV -> electricity conversion
        "eta_ch": 0.70,
        "eta_dis": 0.55,
        "self_dis": 0.0,
        "Emax": 20000.0,             # MWh_HHV
        "Pch_max": 1000.0,           # MW_e into electrolyser
        "Pdis_max": 500.0,           # MW_e out from fuel cell/turbine
        "capex_E": 20000.0,          # €/MWh_HHV storage (placeholder)
        "capex_Pch": 500000.0,       # €/MW_e electrolysis (placeholder)
        "capex_Pdis": 700000.0,      # €/MW_e power block (placeholder)
        "fom_E": 300.0,              # €/MWh_HHV/year (placeholder)
        "fom_P": 15000.0,            # €/MW/year (placeholder)
        "vom_ch": 1.0,
        "vom_dis": 3.0,
        "subsidy_on_discharge_eur_per_MWh_e": 0.0,
        # Apply subsidy on produced H2_HHV over 15 years (from your doc)
        "subsidy_on_h2_MWh_HHV": h2_sub_eur_per_MWh_HHV,
        "capex_grant_per_MWh_store": 0.0,
        "capex_grant_cap_total": 0.0,
        "capex_discount_factor": 0.0,
        "salvage_frac": 0.05,
    },
}

# Economic settings (fill with course-consistent assumptions)
discount_rate = 0.08
H_years = 25
subsidy_years = 15
PW_op = pw_factor(discount_rate, H_years)
PW_sub = pw_factor(discount_rate, min(H_years, subsidy_years))


In [5]:
def solve_one_tech(
    tech_name: str,
    prices: np.ndarray,            # shape (S,T)
    surplus: np.ndarray,           # shape (S,T)
    scenario_prob: np.ndarray,     # shape (S,)
    dt: float,
    year_scale: float,
    discount_rate: float,
    H_years: int,
    subsidy_years: int,
    output_flag: int = 0,
    time_limit: float | None = None,
    prevent_simultaneous: bool = True,   
):
    td = TECH[tech_name]
    prices = as_2d(prices)
    surplus = as_2d(surplus)
    S, T = prices.shape

    PW_op = pw_factor(discount_rate, H_years)
    PW_sub = pw_factor(discount_rate, min(H_years, subsidy_years))

    m = gp.Model(f"storage_{tech_name}")
    m.Params.OutputFlag = output_flag
    if time_limit is not None:
        m.Params.TimeLimit = time_limit

    # If we add binaries, this becomes a MILP. These settings keep it stable/fast for your size.
    if prevent_simultaneous:
        m.Params.MIPGap = 0.05

    # -------------------------
    # Design vars
    # -------------------------
    E = m.addVar(lb=0.0, ub=td["Emax"], name="E")                 # MWh_store (or MWh_HHV for hydrogen)
    Pch = m.addVar(lb=0.0, ub=td["Pch_max"], name="Pch")          # MW
    Pdis = m.addVar(lb=0.0, ub=td["Pdis_max"], name="Pdis")       # MW

    # Optional one-time CAPEX grant (linearised min(rate*E, cap))
    grant = m.addVar(lb=0.0, ub=td["capex_grant_cap_total"], name="capex_grant")
    if td["capex_grant_cap_total"] > 0 and td["capex_grant_per_MWh_store"] > 0:
        m.addConstr(grant <= td["capex_grant_per_MWh_store"] * E, name="grant_rate")
        m.addConstr(grant <= td["capex_grant_cap_total"], name="grant_cap")
    else:
        m.addConstr(grant == 0.0, name="no_grant")

    # -------------------------
    # Operational vars
    # -------------------------
    c = m.addVars(S, T, lb=0.0, name="c")       # MW charging
    d = m.addVars(S, T, lb=0.0, name="d")       # MW discharging
    soc = m.addVars(S, T+1, lb=0.0, name="soc") # MWh state of charge

    # NEW: mode binaries to prevent simultaneous charge/discharge
    # z[s,t] = 1 => charging allowed, discharging forced to 0
    # z[s,t] = 0 => discharging allowed, charging forced to 0
    if prevent_simultaneous:
        z = m.addVars(S, T, vtype=GRB.BINARY, name="isCharging")
    else:
        z = None

    eta_ch = td["eta_ch"]
    eta_dis = td["eta_dis"]
    self_dis = td["self_dis"]

    # Big-M constants (must be constants, not variables)
    M_ch = float(td["Pch_max"])
    M_dis = float(td["Pdis_max"])

    # -------------------------
    # Constraints
    # -------------------------
    for s in range(S):
        m.addConstr(soc[s, 0] <= E, name=f"soc0_ub[{s}]")
        m.addConstr(soc[s, 0] >= 0.0, name=f"soc0_lb[{s}]")
        m.addConstr(soc[s, T] == soc[s, 0], name=f"cyclic[{s}]")

        for t in range(T):
            m.addConstr(soc[s, t] <= E, name=f"soc_ub[{s},{t}]")

            # Power bounds by installed capacities
            m.addConstr(c[s, t] <= Pch, name=f"c_ub_by_Pch[{s},{t}]")
            m.addConstr(d[s, t] <= Pdis, name=f"d_ub_by_Pdis[{s},{t}]")

            # Charging limited by available surplus/curtailment
            m.addConstr(c[s, t] <= float(surplus[s, t]), name=f"surplus[{s},{t}]")

            # NEW: forbid simultaneous charging and discharging
            if prevent_simultaneous:
                m.addConstr(c[s, t] <= M_ch * z[s, t], name=f"noSim_c[{s},{t}]")
                m.addConstr(d[s, t] <= M_dis * (1 - z[s, t]), name=f"noSim_d[{s},{t}]")

            # SOC dynamics
            m.addConstr(
                soc[s, t+1]
                ==
                (1.0 - self_dis) * soc[s, t]
                + eta_ch * c[s, t] * dt
                - (d[s, t] * dt) / eta_dis,
                name=f"dyn[{s},{t}]"
            )

    # -------------------------
    # Economics
    # -------------------------
    capex_raw = td["capex_E"] * E + td["capex_Pch"] * Pch + td["capex_Pdis"] * Pdis
    capex_eff = (1.0 - td["capex_discount_factor"]) * capex_raw - grant

    fom = td["fom_E"] * E + td["fom_P"] * (Pch + Pdis)

    sub_on_dis = td["subsidy_on_discharge_eur_per_MWh_e"]
    sub_on_h2  = td["subsidy_on_h2_MWh_HHV"]

    exp_annual_cash = gp.LinExpr()
    for s in range(S):
        prob = float(scenario_prob[s])
        scen_expr = gp.LinExpr()

        for t in range(T):
            price = float(prices[s, t])

            # arbitrage revenue: sell d, buy c
            scen_expr += year_scale * (price * (d[s, t] - c[s, t]) * dt)

            # variable O&M
            scen_expr += year_scale * (-(td["vom_ch"] * c[s, t] * dt + td["vom_dis"] * d[s, t] * dt))

            # subsidy on discharge (if any)
            if sub_on_dis != 0.0:
                scen_expr += year_scale * (sub_on_dis * d[s, t] * dt)

            # subsidy on H2 produced (if any): eta_ch * charged electricity
            if sub_on_h2 != 0.0:
                scen_expr += year_scale * (sub_on_h2 * (eta_ch * c[s, t] * dt))

        scen_expr += -fom
        exp_annual_cash += prob * scen_expr

    # subsidy-only part for correct 15y vs H horizon discounting
    exp_annual_subsidy = gp.LinExpr()
    for s in range(S):
        prob = float(scenario_prob[s])
        sub_expr = gp.LinExpr()
        for t in range(T):
            if sub_on_dis != 0.0:
                sub_expr += year_scale * (sub_on_dis * d[s, t] * dt)
            if sub_on_h2 != 0.0:
                sub_expr += year_scale * (sub_on_h2 * (eta_ch * c[s, t] * dt))
        exp_annual_subsidy += prob * sub_expr

    salvage = td["salvage_frac"] * capex_eff
    salvage_pv = salvage / ((1.0 + discount_rate) ** H_years)

    NPV = -capex_eff + PW_op * exp_annual_cash + (PW_sub - PW_op) * exp_annual_subsidy + salvage_pv
    m.setObjective(NPV, GRB.MAXIMIZE)
    m.optimize()

    if m.Status not in (GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.SUBOPTIMAL):
        return {"tech": tech_name, "status": int(m.Status), "npv": None}

    E_val = float(E.X)
    Pch_val = float(Pch.X)
    Pdis_val = float(Pdis.X)
    npv_val = float(m.ObjVal)

    c0 = np.array([c[0, t].X for t in range(T)])
    d0 = np.array([d[0, t].X for t in range(T)])
    soc0 = np.array([soc[0, t].X for t in range(T+1)])
    soc_pct = (soc0[:-1] / E_val * 100.0) if E_val > 1e-9 else np.zeros(T)

    out = {
        "tech": tech_name,
        "status": int(m.Status),
        "npv": npv_val,
        "E": E_val,
        "Pch": Pch_val,
        "Pdis": Pdis_val,
        "dispatch_s0": pd.DataFrame({
            "t": np.arange(T),
            "price": prices[0],
            "surplus": surplus[0],
            "charge_MW": c0,
            "discharge_MW": d0,
            "soc_MWh": soc0[:-1],
            "soc_pct": soc_pct,          # NEW
        }),
    }
    

    # Optional quick diagnostic: overlap should be zero by construction when prevent_simultaneous=True
    if prevent_simultaneous:
        out["simultaneous_hours_s0"] = int(np.sum((c0 > 1e-6) & (d0 > 1e-6)))

    return out

In [6]:
results = []
for tech_name in TECH.keys():
    res = solve_one_tech(
        tech_name=tech_name,
        prices=prices,
        surplus=surplus,
        scenario_prob=scenario_prob,
        dt=dt,
        year_scale=year_scale,
        discount_rate=discount_rate,
        H_years=H_years,
        subsidy_years=subsidy_years,
        output_flag=1,
        time_limit=60.0,
        prevent_simultaneous=True,  # <-- NEW
    )
    results.append(res)

summary = pd.DataFrame([{
    "tech": r["tech"],
    "status": r["status"],
    "NPV": r["npv"],
    "E (MWh_store)": r.get("E", np.nan),
    "Pch (MW)": r.get("Pch", np.nan),
    "Pdis (MW)": r.get("Pdis", np.nan),
    "simultaneous_hours_s0": r.get("simultaneous_hours_s0", np.nan),
} for r in results]).sort_values("NPV", ascending=False)

summary

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2748214
Academic license 2748214 - for non-commercial use only - registered to j.___@student.utwente.nl
Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8352V CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 72 physical cores, 144 logical processors, using up to 32 threads

Non-default parameters:
TimeLimit  60
MIPGap  0.05

Academic license 2748214 - for non-commercial use only - registered to j.___@student.utwente.nl
Optimize a model with 306616 rows, 175209 columns and 657026 nonzeros
Model fingerprint: 0xeb73fc15
Variable types: 131409 continuous, 43800 integer (43800 binary)
Coefficient statistics:
  Matrix range     [9e-01, 5e+02]
  Objective range  [2e-02, 2e+05]
  Bounds range     [1e+00, 5e+03]
  RHS r

,tech,status,NPV,E (MWh_store),Pch (MW),Pdis (MW),simultaneous_hours_s0
0,PHS,9,2.135353e+09,2782.330113,500.000000,500.000000,0
2,Battery,2,7.972976e+06,954.564244,335.102346,452.851442,0
1,Flywheel,2,-0.000000e+00,0.000000,0.000000,0.000000,0
3,Hydrogen,9,-0.000000e+00,0.000000,0.000000,0.000000,0


In [7]:
# =========================
# 5) Best alternative + its dispatch (scenario 0)
# =========================
best = max([r for r in results if r["npv"] is not None], key=lambda x: x["npv"])
best["tech"], best["npv"]


('PHS', 2135353402.7568398)

In [8]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df = best["dispatch_s0"]

t = df["t"].to_numpy()
price = df["price"].to_numpy()
charge = df["charge_MW"].to_numpy()
discharge = df["discharge_MW"].to_numpy()

soc_mwh = df["soc_MWh"].to_numpy()
soc_pct = df["soc_pct"].to_numpy()

t_min = int(np.nanmin(t))
t_max = int(np.nanmax(t))

default_start = max(t_min, 0)
default_end = min(t_max, default_start + 24)
if default_end <= default_start:
    default_end = min(t_max, default_start + 1)

def build_fig():
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.06,
        subplot_titles=(
            f"Scenario 0 price - {best['tech']}",
            f"Charge / discharge - {best['tech']}",
            f"State of charge - {best['tech']}"
        )
    )

    fig.add_trace(go.Scatter(x=t, y=price, mode="lines", name="Price (€/MWh)"), row=1, col=1)
    fig.add_trace(go.Scatter(x=t, y=charge, mode="lines", name="Charge (MW)"), row=2, col=1)
    fig.add_trace(go.Scatter(x=t, y=discharge, mode="lines", name="Discharge (MW)"), row=2, col=1)
    fig.add_trace(go.Scatter(x=t, y=soc_pct, mode="lines", name="SOC (%)"), row=3, col=1)

    fig.update_yaxes(title_text="€/MWh", row=1, col=1)
    fig.update_yaxes(title_text="MW", row=2, col=1)
    fig.update_yaxes(title_text="Energy", row=3, col=1)
    fig.update_yaxes(title_text="SOC (%)", row=3, col=1, range=[0, 100])
    fig.update_xaxes(title_text="t", row=3, col=1)

    fig.update_layout(
        hovermode="x unified",
        height=850,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    )

    # Optional: range slider on bottom axis (still useful in addition to manual range)
    fig.update_xaxes(rangeslider_visible=True, row=3, col=1)
    return fig

# ---- Controls (start/end inputs + sliders) ----
start_box = widgets.IntText(value=default_start, description="start t")
end_box   = widgets.IntText(value=default_end, description="end t")

start_slider = widgets.IntSlider(
    value=default_start, min=t_min, max=t_max-1, step=1,
    description="start", continuous_update=False, layout=widgets.Layout(width="700px")
)
end_slider = widgets.IntSlider(
    value=default_end, min=t_min+1, max=t_max, step=1,
    description="end", continuous_update=False, layout=widgets.Layout(width="700px")
)

out = widgets.Output()
_lock = {"busy": False}

def clamp_and_sync(start, end):
    start = int(max(t_min, min(start, t_max - 1)))
    end   = int(max(start + 1, min(end, t_max)))

    # keep end slider feasible
    end_slider.min = start + 1
    if end < end_slider.min:
        end = end_slider.min

    return start, end

def redraw(start, end):
    fig = build_fig()
    fig.update_xaxes(range=[start, end])  # applies to all shared x-axes
    with out:
        clear_output(wait=True)
        display(fig)

def on_any_change(_):
    if _lock["busy"]:
        return
    _lock["busy"] = True
    try:
        # take values from boxes (authoritative)
        start, end = clamp_and_sync(start_box.value, end_box.value)

        # sync everything
        start_box.value = start
        end_box.value = end
        start_slider.value = start
        end_slider.value = end

        redraw(start, end)
    finally:
        _lock["busy"] = False

def on_slider_change(_):
    if _lock["busy"]:
        return
    _lock["busy"] = True
    try:
        start, end = clamp_and_sync(start_slider.value, end_slider.value)

        start_box.value = start
        end_box.value = end
        start_slider.value = start
        end_slider.value = end

        redraw(start, end)
    finally:
        _lock["busy"] = False

start_box.observe(on_any_change, names="value")
end_box.observe(on_any_change, names="value")
start_slider.observe(on_slider_change, names="value")
end_slider.observe(on_slider_change, names="value")

# Initial draw
start0, end0 = clamp_and_sync(default_start, default_end)
start_box.value, end_box.value = start0, end0
start_slider.value, end_slider.value = start0, end0
redraw(start0, end0)

display(widgets.VBox([
    widgets.HBox([start_box, end_box]),
    start_slider,
    end_slider,
    out
]))

In [9]:
td = TECH["Hydrogen"]
df = best["dispatch_s0"]
year_scale = 8760/(T*dt)

annual_charge_MWh = year_scale * (df["charge_MW"].to_numpy() * dt).sum()
annual_h2_MWh_HHV = td["eta_ch"] * annual_charge_MWh
annual_h2_subsidy = TECH["Hydrogen"]["subsidy_on_h2_MWh_HHV"] * annual_h2_MWh_HHV

annual_charge_MWh, annual_h2_MWh_HHV, annual_h2_subsidy


(2029381.2265387066, 1420566.8585770945, 539247179.515865)